In [1]:
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect("salaries.db")
columns = ["File_Date",
           "SubAgency", 
           "Age_Range",
           "Education_Level",
           "Pay_Plan",
           "Grade",
           "LOS_Level",
           "Occupation",
           "PATCO",
           "Adjusted_Basic_Pay",
           "Supervisory_Status",
           "TOA",
           "Work_Schedule",
           "NSFTP_Indicator"]

In [ ]:
pd.read_sql_query("""
SELECT 
    File_Date, 
    Occupation,
    AVG(Adjusted_Basic_Pay) AS Average_Salary
FROM 
    data 
GROUP BY
    File_Date, 
    Occupation
;""", conn)

In [ ]:
conn.close()